# Quantum Detector Tomography

This notebook contains examples for using QDT module with Qiskit.

## Theoretical background

### Noisy quantum detectors
In quantum mechanics any measurement can be described by Positive Operator-Valued Measure (POVM). 
A POVM $\mathbf{M}$ with $n$ outcomes is a list of $n$ operators

$$
\mathbf{M} = \left(M_0, M_1, \dots, M_{n-1}\right) ,
$$
which fulfill requirements

$$
M_i\geq 0, \quad \sum_i M_i = \mathbf{1} \ ,
$$
for all $i$, with $\mathbf{1}$ being identity on given Hilbert space. 
Each POVM's element $M_i$ corresponds to particular outcome of experiment labeled as $i$.
If we perform measurement $\mathbf{M}$ on the quantum state $\rho$, then the probability of obtaining outcome $i$ is
given by the Born's rule:

$$
p\left(i|\rho,\mathbf{M}\right) = \text{Tr}\left(\rho M_i\right)\ .
$$
Since POVMs can describe arbitrary quantum measurements, we can use this formalism to model noisy detectors. 

### Ideal quantum detectors

If it comes to ideal measurements, often the model for the perfect POVM belongs to the particular subset of all possible
measurements. This subset is known as projective measurements. The POVM $\mathbf{P}$ belonging to this subset consists
of operators which fulfill additional property
$$
P_iP_j = \delta_{i,j} P_i \ ,
$$
for all $i$. 
In particular the model for ideal measurement in, e.g., IBM's, Rigetti's or Google's quantum devices is a projective
measurement in computational basis.
Operators of this measurement are simply of the form
$$
P_i = |i><i| \ , 
$$
where $|i>$ is a vector from standard basis, i.e., the vector with only single element - number 1 on $i$th position.




### Quantum Detector Tomography
Quantum Detector Tomography/Quantum Measurement Tomography is a procedure aiming to obtain classical description (i.e.,
matrix representation) of the POVM implemented in the quantum device. The technique requires preparation of different
quantum states and measuring them with the detector (in analogy to different measurement in quantum state tomography).
Here the implicit assumptions are stability of detector in time, access to infinite number of experiments and perfect
state preparation. The minimal number of distinct experiments required to perform QDT when one uses eigenstates of Pauli
operators is $4^K$ ($K$ being the number of qubits), however sometimes it might be beneficial to use more if possible
[1], [2]. By obtaining POVMs via QDT technique, we can not only properly characterize noisy measurement that is
performed by the device, but we can also use this knowledge to mitigate errors in future experiments (see our
[Error-Mitigation Tutorial](Error_Mitigation_Tutorial.ipynb)).

There are several subroutines of POVMs reconstruction that can be used in QDT. The one we implement is iterative
algorithm from from Ref. [3]. The algorithm provably converges to maximum-likelihood estimator and it asserts positivity
of operators in each iteration step by use of Lagrange multipliers.

### Distances between measurements
Having reconstructed (via QDT) noisy POVM $\mathbf{M}$, we would naturally like to compare it with the ideal detector
$\mathbf{P}$. In order to achieve that, we need some notion of distance between quantum measurements. Distances between
POVMs are usually closely related to the distances between probability distributions that they generate via Born's rule.
Hence first we will need a measure of distance between probability distributions. The one we choose here is a
$\textbf{Total-Variation Distance}$ defined as
$$
D_{TV}\left(\mathbf{p},\mathbf{q}\right) = \frac{1}{2} ||\mathbf{p}-\mathbf{q}||_1 = \frac{1}{2} \sum_i |p_i-q_i| \ .
$$

Now we can define $\textbf{operational distance}$ between quantum measurements:
$$
D_{op}\left(\mathbf{M},\mathbf{P}\right) =  \max_{\rho} D_{TV} \left(p_{\mathbf{M}}, p_{\mathbf{N}}\right) \ , 
$$
where $p_{\mathbf{M}/\mathbf{N}}$ denote probability distributions generated by $\mathbf{M}/\mathbf{N}$ measuring
quantum state $\rho$ (over which we maximize). Hence the above distance is in fact the worst-case (in quantum states)
scenario for how far from each other can land distributions generated by those POVMs.

Since the above definition involves optimization of TV distance over all quantum states, it is hard to calculate in
practice. Fortunately, the same quantity can be expressed in terms of operator norms of particular POVM's elements,
namely
$$
D_{op}\left(\mathbf{M},\mathbf{P}\right) = \max_{X} \sum_{i \in X} ||M_i-P_i|| \ ,
$$
where maximization goes over all possible combinations of indices (see Ref. [4]).



## Single Qubit detector tomography example

In below example it is shown how to implement single-qubit Quantum Detector Tomography (QDT) using our module. First we
import necessary modules from our repository.

In [21]:
import povmtools
import ancillary_functions as anf
import numpy as np

from qiskit import IBMQ, Aer, execute
from qiskit.providers.aer import noise

from quantum_tomography_qiskit import detector_tomography_circuits
from DetectorTomography import DetectorTomographyFitter, QDTCalibrationSetup, join_povms

Now we need to create quantum circuits which will be used to implement QDT. To do so, we need to define indices of 
qubits on which we want to perform tomography. In this example we choose single qubit with label 3.

In [3]:
# choose qubit indices
test_qubit_index = [3]

We will also need kets which defines probe states for the tomography. In this scenario we will use over-complete set of
Pauli's eigenstates, which is available as a constant in our module:

In [4]:
test_probe_kets = povmtools.pauli_probe_eigenkets

Now we call detector_tomography_circuits method, which will generate desired circuits:

In [5]:
test_circuits = detector_tomography_circuits(test_qubit_index, test_probe_kets)

### User-defined probe states

User can define list of single-qubit state vectors which will be used to perform tomography of the detector. In the case
of multi-qubit tomography, only single-qubit set is required - the multi-qubit states are constructed from proper tensor
products of those (see multi-qubit QDT example below).

In order for tomography to work properly, the set of passed qubit state vectors, when mapped onto quantum states (i.e.,
density matrices), must span the space of Hermitian matrices. In other words, it must form an operator basis (it might
be an over-complete basis).

If no probe_kets are provided then over-complete set of Pauli's eigenstates is used:

{|0>, |1>, |X+>, |X->, |Y+>, |Y->}

(note that in the example above, if we did not specify the kets, nothing would have changed)

### Implementation of QDT

After defining tomography circuits, we need to implement them. To do so, we use Qiskit simulator in the standard way:

In [6]:
backend = Aer.get_backend('qasm_simulator')

#define number of measurement repetitions
shots_number = 2000

QDT_job = execute(test_circuits, backend=backend, shots=shots_number)

results = QDT_job.result()

It's useful to know, that these results can be obtained earlier -- there's no need to obtain them every time we want to
perform QDT. It's, however, vital to understand, that the POVM describing the detector may change over time. There's a
possibility, that using previously obtained job when calculating POVM won't allow us to get actual detector
characterization.

With probe kets and job results we can reconstruct POVM describing a detector. We start with instantiating
DetectorTomographyFitter class object:

In [7]:
DTF = DetectorTomographyFitter()

This class contains method called *get_maximum_likelihood_povm_estimator* which returns the classical description of our
detector. This method requires QDTCalibrationSetup to use. QDT calibration setup stores information about the circuits and experiments that were conducted in order to obtain POVM estimator. Generally, the QDTCalibrationSetup object can be created with it's constructor, by providing some data to it (qubits number in the circuits, probe kets and obtained frequencies arrays), but it can also be generated directly from qiskit job results. Given that we have the results, we will use the latter.

In [8]:
calibration_setup = QDTCalibrationSetup.from_qiskit_results([results], test_probe_kets)

Now, we can use this object to generate the POVM estimator.

In [9]:
# We pass list of results (in this case singular) to the method.
ml_povm_estimator = DTF.get_maximum_likelihood_povm_estimator(calibration_setup)

for m_i in ml_povm_estimator:
    print(m_i)

[[9.99975055e-01+6.00036816e-21j 2.16662618e-03+4.50000925e-03j]
 [2.16662618e-03-4.50000925e-03j 2.49449745e-05+4.28479557e-21j]]
[[ 2.49449745e-05-6.75310252e-21j -2.16662618e-03-4.50000925e-03j]
 [-2.16662618e-03+4.50000925e-03j  9.99975055e-01+2.58958486e-18j]]


In this case, the ideal measurement corresponds to single qubit projective measurement in computational basis:

In [10]:
ideal_measurement = povmtools.computational_projectors(d=2)
for Pi in ideal_measurement:
    print(Pi)

[[1. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 1.]]


Let us calculate operational distance between estimator and perfect measurement:

In [11]:
Dop=povmtools.operational_distance_POVMs(ml_povm_estimator,ideal_measurement)
print(Dop)

0.004994494419648762


We see that it is quite small number, but not exactly 0. The reason is that in the execution of circuits we **sample**
from ideal distributions, hence our results are affected by statistical fluctuations.

To see that, let us increase number of shots by factor ~16, which should decrease statistical errors roughly by factor
~4.

In [13]:
backend = Aer.get_backend('qasm_simulator')

#define number of measurement repetitions
shots_number = 2000*16

QDT_job_big_statistics = execute(test_circuits, backend=backend, shots=shots_number)

results_big_statistics = QDT_job_big_statistics.result()

#get estimator
DTF_big_statistics = DetectorTomographyFitter()
big_calibration_setup = QDTCalibrationSetup.from_qiskit_results([results_big_statistics], test_probe_kets)
ml_povm_estimator_big_statistics = DTF.get_maximum_likelihood_povm_estimator(big_calibration_setup)

In [14]:
#calculate distance
Dop_big_statistics=povmtools.operational_distance_POVMs(ml_povm_estimator_big_statistics,ideal_measurement)
print(Dop_big_statistics)

0.0004896941914591769


## Multiple qubit case

In the mutli-qubit QDT the only difference is in defining set of indices:

In [15]:
# choose qubit indices
test_qubit_indices = [3, 1]

In all methods, we use convention that **we number qubits in the Hilbert space in ascending order**. Hence in this
example the qubits will be sorted in such a way that qubit no. 1 is first and qubit no. 3 is second in the two-qubit
Hilbert space.

In [16]:
#Define probe kets. Note that we define them only for single qubit, in the same way as before. 
test_probe_kets = povmtools.pauli_probe_eigenkets

#get circuits
test_circuits_multiple_qubits = detector_tomography_circuits(test_qubit_indices, test_probe_kets)

#define number of repetitions
shots_number=8192

#run job on simulator
backend = Aer.get_backend('qasm_simulator')
QDT_job_multiple_qubits = execute(test_circuits_multiple_qubits, backend=backend, shots=shots_number)
results_multiple_qubits = QDT_job_multiple_qubits.result()

#get estimator
DTF = DetectorTomographyFitter()
multiple_qubit_calibration_setup = QDTCalibrationSetup.from_qiskit_results([results_multiple_qubits], test_probe_kets)
ml_povm_estimator_multiple_qubits = \
    DTF.get_maximum_likelihood_povm_estimator(multiple_qubit_calibration_setup)

Again, let's compare it with perfect measurement:

In [17]:
#get ideal measurement in computational basis
ideal_measurement_multiple_qubits = povmtools.computational_projectors(d=2**len(test_qubit_indices))

#calculate distance
Dop_multiple_qubits = \
    povmtools.operational_distance_POVMs(ml_povm_estimator_multiple_qubits,ideal_measurement_multiple_qubits)
print(Dop_multiple_qubits)

0.0022360692926061587


### Joining sets of qubits

In general, performing tomography on the whole device is infeasible. For 15-qubit IBM's device it would require at least
4^15~1 073 741 824 quantum circuits. It is not possible to run so many circuits on any device (and even if it were
possible, the sampling complexity would be daunting).

However, under constraints on the locality of the correlations between errors in the device, it is possible to perform
restricted QDT.

To see that, let us consider the layout of the 5q devices with T-shape connectivity (a lot of such devices are currently
available in IBM Q Experience).

Let us assume that from some other experiments we have evidence that errors on qubits 0 and 2 do not depend on the
measurements of qubits 1,3,4, but they do depend on each other. This is to be understood that doing tomography
separately on q0 and q2 and joining the results give different results than doing it simultaneously on q0q2. At the same
time, doing tomography on all qubits and taking marginals for pair q0q2 does not change the results.

Let us assume that qubits 1,3,4 also depend on each other (and only on each other) in analogous manner. 

In that case, we can perform two tomographies:

In [19]:
# choose qubit indices
qubit_indices0, qubit_indices1 = [0,2] , [1,3,4]


#Define probe kets. Note that we define them only for single qubit, in the same way as before. 
probe_kets = povmtools.pauli_probe_eigenkets

#get circuits
circuits0 = detector_tomography_circuits(qubit_indices0, test_probe_kets)
circuits1 = detector_tomography_circuits(qubit_indices1, test_probe_kets)

#define number of repetitions
shots_number = 8192

#run first job on simulator and get results
backend = Aer.get_backend('qasm_simulator')
QDT_job0 = execute(circuits0, backend=backend, shots=shots_number)
results0 = QDT_job0.result()

DTF = DetectorTomographyFitter()
job0_calibration_setup = QDTCalibrationSetup.from_qiskit_results([results0], test_probe_kets)
POVM0 = DTF.get_maximum_likelihood_povm_estimator(job0_calibration_setup)

#run second job on simulator and get results
backend = Aer.get_backend('qasm_simulator')
QDT_job1 = execute(circuits1, backend=backend, shots=shots_number)
results1 = QDT_job1.result()

DTF = DetectorTomographyFitter()
job1_calibration_setup = QDTCalibrationSetup.from_qiskit_results([results1], test_probe_kets)
POVM1 = DTF.get_maximum_likelihood_povm_estimator(job1_calibration_setup)

Now we have two tomographies of 4- and 8-dimensional POVMs. We would like to join them into big POVM on 32-dimensional
Hilbert space:

In [22]:
#define list of POVMs:
list_POVMs = [POVM0, POVM1]

#define list of associated qubit indices
list_indices = [qubit_indices0, qubit_indices1]

big_POVM = join_povms(list_POVMs,list_indices);

This POVM is quite big, so it is impractical to print whole matrices. However, to check if the order of effects is
correct, we may print only *i*th diagonal element of *i*th effect. Each such element should be ~1 (if order was
incorrect, we would have sometimes obtained values close to 0):

In [23]:
for i in range(2**(sum([len(x) for x in list_indices]))):
    m_i = big_POVM[i]
    
    #take real part to not print 0j
    print(np.real(np.round(m_i[i,i],5)))

0.99999
0.99998
0.99999
0.99999
0.99999
0.99998
0.99999
0.99999
0.99999
0.99999
0.99999
0.99999
0.99999
0.99999
0.99999
0.99999
0.99998
0.99997
0.99998
0.99998
0.99998
0.99997
0.99998
0.99998
0.99998
0.99998
0.99998
0.99998
0.99999
0.99998
0.99998
0.99998


## References

[1] Maciejewski F. B., __POVM's on quantum devices__, Master's Thesis, 2019, Warsaw.

[2] Maciejewski B. F., Zimboras Z., Oszmaniec M.,
[Mitigation of readout noise in near-term quantum devices by classical post-processing based on detector tomography](https://arxiv.org/abs/1907.08518), 2019.

[3] Hradil Z., Rehacek J., Fiurasek J. and Jezek M., __3 maximum-likelihood methods in quantum mechanics__, Quantum
State Estimation (Paris M., Rehacek J. eds) pp.59-112, Springer, Berlin, 2004.

[4] Zbigniew Puchała, Łukasz Pawela, Aleksandra Krawiec, Ryszard Kukulski, [Strategies for optimal single-shot
discrimination of quantum measurements](https://arxiv.org/abs/1804.05856), Phys. Rev. A 98, 042103, 2018